In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers



In [2]:
class BSTEnvironment:
    def __init__(self, max_tree_size=16):
        self.max_tree_size = max_tree_size

    def reset(self):
        self.state = []
        return np.array(self.state)

    def step(self, states, action):
        new_state = self.apply_action(states, action)
        reward = self.calculate_reward(self.apply_action(state, action))
        done = len(new_state) == self.max_tree_size  # Episode ends when the sequence is complete
        return new_state, reward, done

    def apply_action(self, states, action):
        new_state = np.append(states, action)
        return new_state

    def calculate_reward(self, new_state):
        reward = 1.0 if self._check_property(new_state) else 0.0
        return reward
    def _check_property(self,new_state):
        # Check if the sequence forms a valid BST
        return self._is_bst(new_state)

    def _is_bst(self, sequence):
        if len(sequence) < 3:
            return True 
        last_value = sequence[-1]
        left_subtree = [val for val in sequence if val < last_value]
        right_subtree = [val for val in sequence if val > last_value]
        return self._is_bst(left_subtree) and self._is_bst(right_subtree)


In [3]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder


class RLAgent:
    def __init__(self, input_size, output_size, max_tree_size=16):
        self.max_tree_size = max_tree_size
        self.model = self._build_model(input_size, output_size)
        self.model_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    def _build_model(self, input_size, output_size):
        model = tf.keras.Sequential([
            layers.Dense(64, activation='relu', input_shape=(input_size,)),
            layers.Dense(output_size, activation='softmax')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy')
        return model

    def select_action(self, state):
        action = np.array(np.random.randint(1, 101))
        return action
    def predict_action(self, state):
        print(state)
        ac = []
        for _ in range(self.max_tree_size):
            # Use the model to predict the next action
            action_prob = self.model.predict(np.array(state).reshape(1, -1))
            action = np.random.choice(len(action_prob), p=action_prob)
            ac.append(action)
        return ac

    def train(self, states, actions, rewards):
        discounted_rewards = self._calculate_discounted_rewards(rewards)
        actions_one_hot = tf.one_hot(actions, depth=self.model.output_shape[1])
        advantages = discounted_rewards

    # Ensure the states are properly shaped
        states = np.reshape(states, (1, -1))
        with tf.GradientTape() as tape:
            policy_prob = self.model(states)
            action_prob = tf.reduce_sum(actions_one_hot * policy_prob, axis=1)
            loss = -tf.reduce_sum(tf.math.log(action_prob) * advantages)
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.model_optimizer.apply_gradients(zip(grads, self.model.trainable_variables))


    def _calculate_discounted_rewards(self, rewards, gamma=0.99):
        discounted_rewards = np.zeros_like(rewards, dtype=np.float32)
        running_add = 0
        for t in reversed(range(len(rewards))):
            running_add = running_add * gamma + rewards[t]
            discounted_rewards[t] = running_add
        return discounted_rewards
    def generate_rl_guided_input(self, num_samples=5):
        guided_inputs = []
        for _ in range(num_samples):
            state = self.generate_state()
            action = self.predict_action(state)
            guided_inputs.append(self.apply_action([], action))

        return guided_inputs
    
    def generate_state(self):
        return np.array(random.sample(range(1, 101), self.max_tree_size))

    def apply_action(self, state, action):
        # Apply the action by inserting the chosen action into the state
        new_state = state + [action]
        return new_state



In [4]:
# Instantiate RLAgent with desired input and output sizes
env = BSTEnvironment()
gen_test_inp=[]
for i in range(5):
    inp_size=random.randint(10, 90)
    agent = RLAgent(input_size=inp_size, output_size=101, max_tree_size=inp_size)
    for episode in range(2):
        state = env.reset()
        total_reward = 0
        states,actions,rewards=np.array([]),np.array([]),np.array([])
        while True:
            action = agent.select_action(state)
            next_state, reward, done = env.step(states, action)
            rewards=np.append(rewards,reward)
            actions=np.append(actions,action)
            states= next_state
        # Update the agent with the experience
            total_reward += reward
            if done:
                break
        gen_test_inp.append(states)
    

In [5]:
class TreeNode:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

def construct_bst(root, values):
    for value in values:
        root = insert_into_bst(root, value)
    return root

def insert_into_bst(root, value):
    if root is None:
        return TreeNode(value)

    if value < root.value:
        root.left = insert_into_bst(root.left, value)
    elif value > root.value:
        root.right = insert_into_bst(root.right, value)

    return root

def display_bst(root, level=0, prefix="Root: "):
    if root is not None:
        print(" " * (level * 4) + prefix + str(root.value))
        if root.left is not None or root.right is not None:
            display_bst(root.left, level + 1, "L--- ")
            display_bst(root.right, level + 1, "R--- ")

for i in range(5):
    bst_root = None
    input_data=gen_test_inp[i]
    for value in input_data: 
        bst_root = insert_into_bst(bst_root, value)
    print("Constructed Binary Search Tree:")
    display_bst(bst_root)


Constructed Binary Search Tree:
Root: 26.0
    L--- 2.0
        R--- 13.0
            L--- 8.0
                R--- 12.0
            R--- 18.0
                R--- 19.0
    R--- 94.0
        L--- 55.0
            L--- 27.0
                R--- 33.0
                    R--- 51.0
            R--- 56.0
                R--- 77.0
        R--- 96.0
Constructed Binary Search Tree:
Root: 13.0
    L--- 3.0
    R--- 59.0
        L--- 27.0
            L--- 25.0
            R--- 31.0
                R--- 40.0
                    R--- 43.0
                        R--- 58.0
        R--- 92.0
            L--- 63.0
                R--- 82.0
                    L--- 81.0
                        L--- 74.0
                    R--- 86.0
Constructed Binary Search Tree:
Root: 75.0
    L--- 30.0
        L--- 10.0
            L--- 2.0
                R--- 4.0
                    R--- 6.0
            R--- 25.0
                L--- 17.0
                    L--- 11.0
                R--- 29.0
        R--- 44.0
 